In [1]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False
def get_params_to_update(net, feature_extract):
    params_to_update = net.parameters()
    print("Params to learn:")
    if feature_extract:
        params_to_update = []
        for name,param in net.named_parameters():
            if param.requires_grad == True:
                params_to_update.append(param)
                print("\t",name)
    else:
        for name,param in net.named_parameters():
            if param.requires_grad == True:
                print("\t",name)
    return params_to_update

def make_alexnet(out_features, feature_extract = False):
    net = models.alexnet(pretrained=True)
    set_parameter_requires_grad(net, feature_extract)
    num_ftrs = net.classifier[6].in_features
    net.classifier[6] = nn.Linear(num_ftrs, out_features)
    params_to_update = get_params_to_update(net, feature_extract)
    optimizer = optim.Adam(params_to_update, lr=0.0001)
    return net, optimizer

def make_mobilenet_v3(out_features, feature_extract = False):
    net = models.mobilenet_v3_small(pretrained=True)
    set_parameter_requires_grad(net, feature_extract)
    num_ftrs = net.classifier[3].in_features
    net.classifier[3] = nn.Linear(num_ftrs, out_features)
    params_to_update = get_params_to_update(net, feature_extract)
    optimizer = optim.Adam(params_to_update, lr=0.0001)
    return net, optimizer

def make_inception_v3(out_features, feature_extract = False):
    net = models.inception_v3(pretrained=True)
    set_parameter_requires_grad(net, feature_extract)
    num_ftrs = net.AuxLogits.fc.in_features
    net.AuxLogits.fc = nn.Linear(num_ftrs, out_features)
    num_ftrs = net.fc.in_features
    net.fc = nn.Linear(num_ftrs, out_features)
    params_to_update = get_params_to_update(net, feature_extract)
    optimizer = optim.Adam(params_to_update, lr=0.0001)
    return net, optimizer

In [2]:
import statistics
import torch.utils.data as td
import net_training
from birds_dataset import Birds270Dataset
import torch
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import datasets, models
import pandas as pd

# Splits a dataset randomly into a train and test set. The size of test set is 80% of the whole dataset.
# Then it trains the network
def train_net_random_dataset_split(net, dataset, epochs, optimizer, batch_size, early_stopping, is_inception):
    train_set_size = int(len(dataset)*0.8)
    test_set_size = len(dataset)-train_set_size
    train_dataset, test_dataset = td.random_split(dataset, [train_set_size, test_set_size])
    label_set = dataset.get_label_set()
    train_dataloader = DataLoader(train_dataset, batch_size, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size, shuffle=True)
    device = None # Wybiera CUDA jeśli jest dostępne, w przeciwnym wypadku CPU
    # device = "cpu" # Odkomentować jeżeli CUDA nie będzie działać
    results = net_training.train_and_evaluate(net, train_dataloader, test_dataloader, label_set,early_stopping = early_stopping,
                                 epochs=epochs, optimizer=optimizer, print_results=True, device=device, is_inception=is_inception)
    return results

# Creates a few networks and trains them using a random split of the dataset.
# The number of created networks is in the "repeat" argument
# It returns the final validation results for each network
def cross_validate_net(net_generator, dataset, repeat=5, epochs=20, batch_size=32, is_inception=False):
    all_results = []
    for i in range(repeat):
        print(f"Training network {i+1} ...")
        net, optimizer = net_generator()
        early_stopping = net_training.EarlyStoppingByAccuracy(patience=10)
        results = train_net_random_dataset_split(net, dataset, epochs,optimizer=optimizer,
                                                 early_stopping=early_stopping, batch_size=batch_size, is_inception=is_inception)
        all_results.append(results)
        net_training.print_final_results(results)
        del optimizer, net
        torch.cuda.empty_cache()
    return all_results
    
    
def print_validation_results(cross_validation_results):
    losses = [r["loss"] for r in cross_validation_results]
    accuracies = [r["accuracy"] for r in cross_validation_results]
    print("Losses: ", losses)
    print("Loss:  mean: {:.4f}, std: {:.4f}".format(statistics.mean(losses), statistics.stdev(losses)))
    print("Accuracies: ", accuracies)
    print("Accuracy:  mean: {:.4f}, std: {:.4f}".format(statistics.mean(accuracies), statistics.stdev(accuracies)))
    
def results_to_dataframe(cross_validation_results):
    normalized = pd.json_normalize(cross_validation_results)
    return normalized
    


In [3]:
dataset_dir = "../data/birds270"
selected_birds = ["ALBATROSS", "BALD EAGLE", "BARN OWL", "EURASIAN MAGPIE", "FLAMINGO",
                  "MALLARD DUCK", "OSTRICH", "PEACOCK", "PELICAN", "TRUMPTER SWAN"]

#transform = transforms.Normalize((127.5, 127.5, 127.5), (127.5, 127.5, 127.5)) # normalizes colors to range [-1,1]
transform =  transforms.Compose([
    transforms.Normalize((0, 0, 0), (255, 255, 255)),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

transform_inception =  transforms.Compose([
    transforms.Resize([299, 299]),
    transforms.Normalize((0, 0, 0), (255, 255, 255)),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

#dataset = Birds270Dataset(dataset_dir,  selected_birds=selected_birds, transform=transform)
dataset = Birds270Dataset(dataset_dir,  selected_birds=selected_birds, transform=transform_inception)

def net_generator():
    return make_inception_v3(out_features=len(selected_birds), feature_extract = False)

results = cross_validate_net(net_generator, dataset, repeat=5, epochs=100, batch_size=64, is_inception=True)
print_validation_results(results)

Training network 1 ...
Params to learn:
	 Conv2d_1a_3x3.conv.weight
	 Conv2d_1a_3x3.bn.weight
	 Conv2d_1a_3x3.bn.bias
	 Conv2d_2a_3x3.conv.weight
	 Conv2d_2a_3x3.bn.weight
	 Conv2d_2a_3x3.bn.bias
	 Conv2d_2b_3x3.conv.weight
	 Conv2d_2b_3x3.bn.weight
	 Conv2d_2b_3x3.bn.bias
	 Conv2d_3b_1x1.conv.weight
	 Conv2d_3b_1x1.bn.weight
	 Conv2d_3b_1x1.bn.bias
	 Conv2d_4a_3x3.conv.weight
	 Conv2d_4a_3x3.bn.weight
	 Conv2d_4a_3x3.bn.bias
	 Mixed_5b.branch1x1.conv.weight
	 Mixed_5b.branch1x1.bn.weight
	 Mixed_5b.branch1x1.bn.bias
	 Mixed_5b.branch5x5_1.conv.weight
	 Mixed_5b.branch5x5_1.bn.weight
	 Mixed_5b.branch5x5_1.bn.bias
	 Mixed_5b.branch5x5_2.conv.weight
	 Mixed_5b.branch5x5_2.bn.weight
	 Mixed_5b.branch5x5_2.bn.bias
	 Mixed_5b.branch3x3dbl_1.conv.weight
	 Mixed_5b.branch3x3dbl_1.bn.weight
	 Mixed_5b.branch3x3dbl_1.bn.bias
	 Mixed_5b.branch3x3dbl_2.conv.weight
	 Mixed_5b.branch3x3dbl_2.bn.weight
	 Mixed_5b.branch3x3dbl_2.bn.bias
	 Mixed_5b.branch3x3dbl_3.conv.weight
	 Mixed_5b.branch3x3dbl_3

Epoch 0:
	train loss: 1.6863116861207612
	validation loss: 0.276498260563367, validation accuracy: 99.31506849315068%
	Elapsed time: 0:04:16.556901
Epoch 1:
	train loss: 0.19387999399346306
	validation loss: 0.05470499023795128, validation accuracy: 99.65753424657534%
	Elapsed time: 0:04:10.630764
Epoch 2:
	train loss: 0.04653385406804126
	validation loss: 0.029112840581634273, validation accuracy: 99.65753424657534%
	Elapsed time: 0:04:01.747817
Epoch 3:
	train loss: 0.026253377542692025
	validation loss: 0.024368234820765992, validation accuracy: 99.65753424657534%
	Elapsed time: 0:04:18.714651
Epoch 4:
	train loss: 0.019293291158782474
	validation loss: 0.020045571003670562, validation accuracy: 99.65753424657534%
	Elapsed time: 0:04:02.026211
Epoch 5:
	train loss: 0.018099600720773636
	validation loss: 0.02007789906012277, validation accuracy: 99.31506849315068%
	Elapsed time: 0:04:05.254327
Epoch 6:
	train loss: 0.012329963369751876
	validation loss: 0.02338856428045116, validatio

Epoch 0:
	train loss: 1.6321194102383805
	validation loss: 0.2815958081859432, validation accuracy: 100.0%
	Elapsed time: 0:04:23.192037
Epoch 1:
	train loss: 0.1795403290265216
	validation loss: 0.0486141060262102, validation accuracy: 100.0%
	Elapsed time: 0:04:12.129097
Epoch 2:
	train loss: 0.03772682377087722
	validation loss: 0.023958472458467092, validation accuracy: 100.0%
	Elapsed time: 0:04:13.183779
Epoch 3:
	train loss: 0.023508948051909027
	validation loss: 0.019225136032455587, validation accuracy: 100.0%
	Elapsed time: 0:04:24.396053
Epoch 4:
	train loss: 0.01696763870084265
	validation loss: 0.014094812093838437, validation accuracy: 100.0%
	Elapsed time: 0:04:18.658177
Epoch 5:
	train loss: 0.010284230438912507
	validation loss: 0.012818094419494067, validation accuracy: 100.0%
	Elapsed time: 0:04:21.331602
Epoch 6:
	train loss: 0.009882194100276888
	validation loss: 0.011220577364302662, validation accuracy: 100.0%
	Elapsed time: 0:04:16.883037
Epoch 7:
	train loss: 0

Epoch 0:
	train loss: 1.6237420093645158
	validation loss: 0.2501968690385557, validation accuracy: 100.0%
	Elapsed time: 0:04:15.366360
Epoch 1:
	train loss: 0.1711537107876398
	validation loss: 0.04073618051328071, validation accuracy: 100.0%
	Elapsed time: 0:04:19.234984
Epoch 2:
	train loss: 0.039379195580147144
	validation loss: 0.02000092572137101, validation accuracy: 100.0%
	Elapsed time: 0:04:18.114603
Epoch 3:
	train loss: 0.020749237196676523
	validation loss: 0.014928877430215273, validation accuracy: 100.0%
	Elapsed time: 0:04:10.383968
Epoch 4:
	train loss: 0.013681549074105905
	validation loss: 0.012419566984148058, validation accuracy: 100.0%
	Elapsed time: 0:04:11.269070
Epoch 5:
	train loss: 0.014509604147940516
	validation loss: 0.007873079599174734, validation accuracy: 100.0%
	Elapsed time: 0:04:10.055402
Epoch 6:
	train loss: 0.013934201290942793
	validation loss: 0.02207744195547006, validation accuracy: 99.65753424657534%
	Elapsed time: 0:04:12.192536
Epoch 7:
	

Epoch 0:
	train loss: 1.6089910021368063
	validation loss: 0.2500854742853609, validation accuracy: 99.65753424657534%
	Elapsed time: 0:04:32.204924
Epoch 1:
	train loss: 0.17039334816190432
	validation loss: 0.0472478984969936, validation accuracy: 100.0%
	Elapsed time: 0:04:20.593535
Epoch 2:
	train loss: 0.0425316043977672
	validation loss: 0.03729833749263254, validation accuracy: 99.31506849315068%
	Elapsed time: 0:04:21.579191
Epoch 3:
	train loss: 0.023540140763819727
	validation loss: 0.029232718650098532, validation accuracy: 98.97260273972603%
	Elapsed time: 0:04:29.164642
Epoch 4:
	train loss: 0.01631593180254843
	validation loss: 0.02261621798452449, validation accuracy: 99.31506849315068%
	Elapsed time: 0:04:13.550703
Epoch 5:
	train loss: 0.014718459756153525
	validation loss: 0.020123654108953802, validation accuracy: 99.65753424657534%
	Elapsed time: 0:04:18.052872
Epoch 6:
	train loss: 0.010697622657282274
	validation loss: 0.02059394863676535, validation accuracy: 99.

Epoch 0:
	train loss: 1.6412577836632116
	validation loss: 0.2671695118897582, validation accuracy: 98.97260273972603%
	Elapsed time: 0:05:21.189169
Epoch 1:
	train loss: 0.18959888286799143
	validation loss: 0.05832614014818244, validation accuracy: 99.65753424657534%
	Elapsed time: 0:05:25.629111
Epoch 2:
	train loss: 0.04413547682516775
	validation loss: 0.03834263541518825, validation accuracy: 99.65753424657534%
	Elapsed time: 0:05:27.349479
Epoch 3:
	train loss: 0.02512401369548887
	validation loss: 0.03404950753670849, validation accuracy: 99.65753424657534%
	Elapsed time: 0:05:48.868633
Epoch 4:
	train loss: 0.02130920068068529
	validation loss: 0.03372834311568574, validation accuracy: 99.65753424657534%
	Elapsed time: 0:05:21.160633
Epoch 5:
	train loss: 0.013921825914910481
	validation loss: 0.03420005667291275, validation accuracy: 99.31506849315068%
	Elapsed time: 0:05:20.966652
Epoch 6:
	train loss: 0.012785999305661301
	validation loss: 0.02749813546439352, validation ac

In [4]:
dataframe = results_to_dataframe(results)
dataframe.to_csv("../results/inception_v3_pretrained.csv")